# Keras框架实现mnist手写字体的识别

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

Using TensorFlow backend.


## 1 数据获取及处理
    a>通过keras自建模块导入数据
    b>训练集和测试集需要reshape，channels在前，-1表示不论多少样本
    c>将标签准换为one-hot模式，即0-1二进制类（图片上的数字位置为1，其余为0）
    d>对数据归一化处理，方便使用梯度下降法时更快收敛（需要先将uint8转化为float类型）

In [2]:
(x_train,y_train),(x_test,y_test) = mnist.load_data()

C:\Users\visitor\.keras\datasets\mnist.npz


In [3]:
x_train = x_train.reshape(-1,1,28,28)
x_test = x_test.reshape(-1,1,28,28)
y_train = np_utils.to_categorical(y_train,num_classes=10)
y_test = np_utils.to_categorical(y_test,num_classes=10)

In [4]:
print(x_train.dtype, x_test.dtype)

uint8 uint8


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/=255
x_test/=255

## 2 构建模型
    Conv-->MaxPool-->Conv-->MaxPool-->FC-->Softmax
    激活函数选用'relu'

In [6]:
model = Sequential()

In [7]:
model.add(Convolution2D(data_format='channels_first',filters=32,kernel_size=5,padding='same',input_shape=(1,28,28)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2,strides=2,padding='same',data_format='channels_first'))

In [8]:
model.add(Convolution2D(64,5,strides=1,padding='same',data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2,'same',data_format='channels_first'))

In [9]:
model.add(Flatten()) # 将输入展平，不影响批量大小
model.add(Dense(10))
model.add(Activation('softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
# 打印模型（模型的参数变化情况）
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 28, 28)        832       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 28, 28)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 14, 14)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 14, 14)        51264     
_________________________________________________________________
activation_2 (Activation)    (None, 64, 14, 14)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 7, 7)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

## 3 训练模型
    a>使用Adam来优化
    b>使用tensorboard来实现可视化
    c>精度、损失与训练次数的曲线图见images

In [ ]:
adam = Adam(lr=0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=20,batch_size=128,callbacks=[TensorBoard(log_dir='my_tensorboard')])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/20
60000/60000 [==============================] - 7s - loss: 0.2159 - acc: 0.9370     
Epoch 2/20
60000/60000 [==============================] - 5s - loss: 0.0584 - acc: 0.9813     
Epoch 3/20
60000/60000 [==============================] - 4s - loss: 0.0402 - acc: 0.9875     
Epoch 4/20
60000/60000 [==============================] - 4s - loss: 0.0312 - acc: 0.9904     
Epoch 5/20
60000/60000 [==============================] - 5s - loss: 0.0252 - acc: 0.9914     
Epoch 6/20
60000/60000 [==============================] - 4s - loss: 0.0201 - acc: 0.9932     - ETA: 1s - loss: 0.0202 - acc - ETA: 1s 
Epoch 7/20
60000/60000 [==============================] - 4s - loss: 0.0169 - acc: 0.9946     
Epoch 8/20
60000/60000 [==============================] - 5s - loss: 0.0136 - acc: 0.9957     - ETA: 0s - loss: 0.012
Epoch 9/20
60000/60000 [======

## 4 对测试集进行评估

In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)# 默认batch_size的的大小为32
print('\\ntest loss:',loss)
print('\\ntest accuracy',accuracy)

如果觉得模型效果很好，可保存，下次可以直接导入，对测试集评估

In [ ]:
model.save('keras-mnist-cnn-first.h5')